In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator

import matplotlib.pyplot as plt

import numpy as np

## Fourier Transform

In [ ]:
def quantum_fourier_transform(n):
    quantum_register = QuantumRegister(size=n, name="x")
    QFT_circuit = QuantumCircuit(quantum_register, name=f"QFT")

    

    for i, q in enumerate(quantum_register, start=1):
        QFT_circuit.h(q)
        for j, p in enumerate(quantum_register[i:], start=1):
            QFT_circuit.cp(np.pi / (1 << j), q, p) # I don't understand. Should q not be target and control be p instead? Symmetric so does not matter.

    for q, p in zip(quantum_register[:n >> 1], reversed(quantum_register[n >> 1:])):
        QFT_circuit.swap(q, p)
    
    return QFT_circuit

def inverse_quantum_fourier_transform(n):
    quantum_register = QuantumRegister(size=n, name="x")
    inverse_QFT_circuit = QuantumCircuit(quantum_register, name=f"IQFT")

    for q, p in zip(quantum_register[:n >> 1], reversed(quantum_register[n >> 1:])):
        inverse_QFT_circuit.swap(q, p)
        
    for i, q in enumerate(reversed(quantum_register), start=1):
        for j, p in enumerate(reversed(quantum_register[n + 1 - i:]), start=1):
            inverse_QFT_circuit.cp(- np.pi / (1 << (i - j)), q, p)
        inverse_QFT_circuit.h(q)

    

    return inverse_QFT_circuit

In [1]:
#Check that QFT works
qc_a2 = QuantumRegister(size=1, name="a2")
qc_a1 = QuantumRegister(size=1, name="a1")


#QFT_circuit = QuantumCircuit(qc_c2, qc_c1, qc_b2, qc_b1, qc_a2, qc_a1, name="QFT circuit")
QFT_circuit = QuantumCircuit(qc_a1, qc_a2,name="QFT circuit")
QFT_circuit.compose(quantum_fourier_transform(2), qubits = [1,0],inplace=True)
QFT_circuit.barrier()
QFT_circuit.compose(inverse_quantum_fourier_transform(2), qubits = [1,0],inplace=True)
#QFT_circuit.swap(0,1)
QFT_circuit.draw(output="mpl", style="bw")

NameError: name 'QuantumRegister' is not defined

## Quantum Addition

In [ ]:
## Now I want to make this into a function
def QCalc(n):
    qr_a = QuantumRegister(size=n, name="a")
    qr_b = QuantumRegister(size=n, name="b")
    qr_c = QuantumRegister(size=n, name="c")
    
    qc = QuantumCircuit(qr_a,qr_b,qr_c, name=f"QFT")

    #Initial conditions
    qc.x(qr_b[n-2])
    qc.x(qr_c[n-1])
    print(np.nonzero(np.round(np.real(Statevector(qc).data)))[0][0])

    qc.barrier()
    
    #Apply Fourier Transform
    qc.compose(quantum_fourier_transform(n), qubits = qr_a[::-1],inplace=True)

    
    #Apply addition circuit
    qc.barrier()
    for q, p in zip(qr_a[:n >> 1], reversed(qr_a[n >> 1:])):
        qc.swap(q, p)

    for j, p in enumerate(qr_a, start=0): # is this algorithm fully quantum if I use if-else? Yes because this only helps in construction.
        for i in range(n):
            if i >j:
                pass
            else:
                q = qr_b[i]
                r = qr_c[i]
                qc.cp(2*np.pi/2**(j+1-i), p,q)
                qc.cp(2*np.pi/2**(j+1-i), p,r)
            #QFT_circuit.cp(np.pi / (1 << j), p, q)




    for q, p in zip(qr_a[:n >> 1], reversed(qr_a[n >> 1:])):
        qc.swap(q, p)    

    #Apply Inverse Fourier Transform
    qc.barrier()
    qc.compose(inverse_quantum_fourier_transform(n),qubits = qr_a[::-1], inplace=True)
    

    

    return qc
